In [6]:
pip install pandas numpy matplotlib scikit-learn jupyter  


   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------- ----------------------- 4.5/11.0 MB 24.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 27.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ------------------------- -------------- 8.1/12.8 MB 35.9 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 32.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -------------------------------------- - 7.9/8.1 MB 34.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 29.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------- ----- 7.6/8.7 MB 39.0 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 30.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ----------------------------


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import sqlite3
import unidecode
import pandas as pd

try:
    conn = sqlite3.connect("../datos/qll.db")
    print("Conectado correctamente")
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
    print(tables)
finally:
    conn.close()

Conectado correctamente
            name
0  interacciones
1       lectores
2         libros


In [8]:
conn = sqlite3.connect("../datos/qll.db")
interacciones = pd.read_sql_query("SELECT * FROM interacciones", conn)
lectores = pd.read_sql_query("SELECT * FROM lectores", conn)
libros = pd.read_sql_query("SELECT * FROM libros", conn)
conn.close()

In [9]:
df = (interacciones
      .merge(lectores, on="id_lector", how="left")
      .merge(libros, on="id_libro", how="left"))

df.head()

,id_lector,id_libro,fecha,rating,nombre,genero_x,vive_en,nacimiento,titulo,autor,genero_y,editorial,anio_edicion,isbn,resumen,img_src
0,yura,-incluso-el-olvido,hace 8 años,6,yura,Mujer,Paraná - Argentina,1996,",INCLUSO EL OLVIDO.","VITAS, ROMÁN ROLANDO",Lecturas complementarias,DEL CLÉ,2015,9789873755132.0,Historias y personajes que navegan por las pro...,https://quelibroleo.com/images/libros/libro-14...
1,manucasoul,100-anos-de-pulgarcito-la-revista-donde-empezo...,hace 1 año,10,manuel,Hombre,Madrid - España,1952,100 AÑOS DE PULGARCITO. La revista donde empez...,"GUIRAL, ANTONI","Cómics, Novela Gráfica",BRUGUERA,2021,9788402425171.0,Pulgarcito nació en 1921 en un creciente merca...,https://quelibroleo.com/images/libros/libro-16...
2,aketza,1001-comics-que-hay-que-leer-antes-de-morir,hace 8 años,8,aketza,Hombre,Bilbao - España,1981,1001 COMICS QUE HAY QUE LEER ANTES DE MORIR,"GRAVETT, PAUL",Ensayo,GRIJALBO,2012,9788425347702.0,Una guía completa que recopila los mejores cóm...,https://quelibroleo.com/images/libros/GR47702.jpg
3,albertofg1,1080-recetas-de-cocina,hace 4 años,9,alberto fernandez,Hombre,Seixo marín pontevedra - España,1970,1080 RECETAS DE COCINA,"ORTEGA, SIMONE",Cocina,ALIANZA,2023,9788411483582.0,"""1080 recetas de cocina"" es un clásico por exc...",https://quelibroleo.com/images/libros/libro-17...
4,trini,1080-recetas-de-cocina,hace 12 años,10,trinidad sanz,-,Palma de mallorca - España,1959,1080 RECETAS DE COCINA,"ORTEGA, SIMONE",Cocina,ALIANZA,2023,9788411483582.0,"""1080 recetas de cocina"" es un clásico por exc...",https://quelibroleo.com/images/libros/libro-17...


## Limpieza

In [10]:
df = df.drop(columns=["fecha"])
df["anio_edicion"] = pd.to_numeric(df["anio_edicion"], errors="coerce")
df.rename(columns={"genero_x":"genero","genero_y":"genero_literario"}, inplace=True)

In [11]:
df["pais"] = df["vive_en"].str.split("-").str[-1].str.strip()
df["ciudad"] = df["vive_en"].str.split("-").str[0].str.strip()
df = df.drop(columns=["vive_en"])

In [12]:
df.pais.unique()

array(['Argentina', 'España', '', 'Mexico', 'Costa Rica', 'Guatemala',
       '¿?', 'Switzerland, Swiss Confederation',
       'United States of America', 'Uruguay', 'Colombia', 'Venezuela',
       'Chile', 'Peru', "Cote d'Ivoire", 'Saint Barthelemy', 'Ecuador',
       'Niue', 'Brazil', 'El Salvador', 'France, French Republic',
       'United Kingdom', 'Cuba', 'Honduras', 'Panama', 'Germany',
       'Norway', 'Saint Pierre and Miquelon', 'Japan', 'Italy',
       'Portugal, Portuguese Republic', 'Gambia the', 'Puerto Rico',
       'Fiji the Fiji Islands', 'Belgium', 'Netherlands Antilles',
       'Australia', 'Czech Republic', 'Bolivia', 'Dominican Republic',
       'Andorra', 'Afghanistan', 'Paraguay', 'Austria', 'Poland',
       'Denmark', 'Serbia', 'Turkey', 'Canada', 'Greece', 'Swaziland',
       'Slovakia (Slovak Republic)', 'Netherlands the', 'Kiribati',
       'Romania', 'Macao', 'American Samoa', 'Armenia', 'Bulgaria',
       'Micronesia', 'Guyana', 'Angola', None], dtype=object

In [16]:
df["pais"].fillna("desconocido", inplace=True)
df["pais"] = df["pais"].apply(lambda x: unidecode.unidecode(x.strip().lower()))
map_paises = {
    "espana": "españa",
    "argentina": "argentina",
    "mexico": "méxico",
    "uruguay": "uruguay",
    "chile": "chile",
    "peru": "perú",
    "colombia": "colombia",
    "venezuela": "venezuela",
    "costa rica": "costa rica",
    "guatemala": "guatemala",
    "paraguay": "paraguay",
    "bolivia": "bolivia",
    "ecuador": "ecuador",
    "cuba": "cuba",
    "panama": "panamá",
    "honduras": "honduras",
    "nicaragua": "nicaragua",
    "andorra": "andorra",
    "brasil": "brasil",
    "brazil": "brasil",
    "united states of america": "estados unidos",
    "united kingdom": "reino unido",
    "france, french republic": "francia",
    "germany": "alemania",
    "italy": "italia",
    "portugal, portuguese republic": "portugal",
    "switzerland, swiss confederation": "suiza",
    "austria": "austria",
    "belgium": "belgica",
    "denmark": "dinamarca",
    "netherlands the": "paises bajos",
    "netherlands antilles": "paises bajos",
    "czech republic": "republica checa",
    "slovakia (slovak republic)": "eslovaquia",
    "poland": "polonia",
    "romania": "rumania",
    "turkey": "turquia",
    "greece": "grecia",
    "japan": "japon",
    "canada": "canada",
    "australia": "australia",
    "gambia the": "gambia",
    "fiji the fiji islands": "fiyi",
    "micronesia": "micronesia",
    "guyana": "guyana",
    "armenia": "armenia",
    "angola": "angola",
    "american samoa": "samoa americana",
    "saint barthelemy": "san bartolome",
    "saint pierre and miquelon": "san pedro y miquelon",
    "macao": "macau",
    "swaziland": "esuatini",
    "niue": "niue",
    "kiribati": "kiribati",
    "cote d'ivoire": "costa de marfil",
    "¿?": "desconocido",
    "": "desconocido",
    "??":"desconocido"
}

df["pais"] = df["pais"].replace(map_paises)
df.pais.unique()


C:\Users\NB Milagros Irusta\AppData\Local\Temp\ipykernel_13580\1093092711.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["pais"].fillna("desconocido", inplace=True)


array(['argentina', 'españa', 'desconocido', 'méxico', 'costa rica',
       'guatemala', 'suiza', 'estados unidos', 'uruguay', 'colombia',
       'venezuela', 'chile', 'perú', 'costa de marfil', 'san bartolome',
       'ecuador', 'niue', 'brasil', 'el salvador', 'francia',
       'reino unido', 'cuba', 'honduras', 'panamá', 'alemania', 'norway',
       'san pedro y miquelon', 'japon', 'italia', 'portugal', 'gambia',
       'puerto rico', 'fiyi', 'belgica', 'paises bajos', 'australia',
       'republica checa', 'bolivia', 'dominican republic', 'andorra',
       'afghanistan', 'paraguay', 'austria', 'polonia', 'dinamarca',
       'serbia', 'turquia', 'canada', 'grecia', 'esuatini', 'eslovaquia',
       'kiribati', 'rumania', 'macau', 'samoa americana', 'armenia',
       'bulgaria', 'micronesia', 'guyana', 'angola'], dtype=object)

In [18]:
df = df.drop(columns=["ciudad"])

In [22]:
df.isna().sum()


id_lector           0
id_libro            0
rating              0
nombre              0
genero              0
nacimiento          0
titulo              0
autor               0
genero_literario    0
editorial           0
anio_edicion        0
isbn                0
resumen             0
img_src             0
pais                0
dtype: int64

In [21]:
df.dropna(inplace=True)

In [29]:
def normalizar_generos(generos):
    generos_normalizados = []
    generos_normalizados_dict ={
    'narrativa':'narrativa',
    'ensayo':'ensayo',
    'lecturas complementarias':'ensayo',
    'historia':'historia',
    'histórica y aventuras': 'histórica y aventuras',
    'literatura contemporánea': 'literatura contemporánea',
    'ciencia ficción en bolsillo': 'fantástica, ciencia ficción',
    'no ficción': 'no ficción',
    'infantil y juvenil': 'infantil y juvenil',
    'romántica, erótica': 'romántica',
    'clásicos de la literatura': 'clásicos',
    'fantástica, ciencia ficción': 'fantástica, ciencia ficción',
    'política nacional': 'política',
    'novela negra': 'novela negra, intriga, terror',
    'cómics de humor': 'humor',
    'humor': 'humor',
    'medicina divulgativa': 'medicina',
    'poesía, teatro': 'arte',
    'idiomas': 'idiomas',
    'ciencias políticas y sociales': 'política',
    'autoayuda y espiritualidad': 'autoayuda',
    'autoayuda': 'autoayuda',
    'biografías, memorias': 'no ficción',
    'NO ESPECIFICADO': 'no especificado',
    'novela negra, intriga, terror': 'novela negra, intriga, terror',
    'policiaca. Novela negra en bolsillo': 'novela negra, intriga, terror',
    'negra': 'novela negra, intriga, terror',
    'economía': 'economía',
    'economía financiera': 'economía',
    'estudios y ensayo': 'no ficción',
    'Dietética y nutrición': 'medicina',
    'deportes y juegos': 'deporte',
    'poesía': 'arte',
    'música': 'arte',
    'astrología y horóscopos': 'espiritual',
    'psicología y pedagogía': 'psicologia',
    'medicina': 'medicina',
    'historia moderna de españa': 'no ficción',
    'álbumes ilustrados': 'arte',
    'fotografía': 'arte',
    'didáctica y metodología': 'académico',
    'novela': 'literatura contemporánea',
    'cómics, novela gráfica':'comics',
    'varios':'varios',
    'cómics':'comics',
    'cocina':'cocina',
    'guías de viaje':'no ficción',
    'informática':'informática',
    'historia del cine':'arte',
    'televisión':'televisión',
    'astrología y horóscopos':'astrología',
    'romántica':'romántica',
    'matemáticas divulgativas':'matemáticas',
    'historia moderna de españa':'historia'
    }

    # Iterar sobre cada género y normalizarlo
    for genero in generos:
        genero_normalizado = generos_normalizados_dict.get(genero.lower(), genero.lower())
        generos_normalizados.append(genero_normalizado)

    return generos_normalizados

df['genero_literario']=normalizar_generos(df['genero'])

df.head()

,id_lector,id_libro,rating,nombre,genero,nacimiento,titulo,autor,genero_literario,editorial,anio_edicion,isbn,pais
0,yura,-incluso-el-olvido,6,yura,Mujer,1996,",INCLUSO EL OLVIDO.","VITAS, ROMÁN ROLANDO",mujer,DEL CLÉ,2015.0,9789873755132.0,argentina
1,manucasoul,100-anos-de-pulgarcito-la-revista-donde-empezo...,10,manuel,Hombre,1952,100 AÑOS DE PULGARCITO. La revista donde empez...,"GUIRAL, ANTONI",hombre,BRUGUERA,2021.0,9788402425171.0,españa
2,aketza,1001-comics-que-hay-que-leer-antes-de-morir,8,aketza,Hombre,1981,1001 COMICS QUE HAY QUE LEER ANTES DE MORIR,"GRAVETT, PAUL",hombre,GRIJALBO,2012.0,9788425347702.0,españa
3,albertofg1,1080-recetas-de-cocina,9,alberto fernandez,Hombre,1970,1080 RECETAS DE COCINA,"ORTEGA, SIMONE",hombre,ALIANZA,2023.0,9788411483582.0,españa
4,trini,1080-recetas-de-cocina,10,trinidad sanz,-,1959,1080 RECETAS DE COCINA,"ORTEGA, SIMONE",-,ALIANZA,2023.0,9788411483582.0,españa


## Codificación de variables

In [26]:
df.dtypes

id_lector            object
id_libro             object
rating                int64
nombre               object
genero               object
nacimiento           object
titulo               object
autor                object
genero_literario     object
editorial            object
anio_edicion        float64
isbn                 object
resumen              object
img_src              object
pais                 object
dtype: object

In [27]:
df.drop(columns={"resumen", "img_src"}, inplace=True)

In [36]:
cat_cols = ["genero", "genero_literario", "pais"]
df_encoded = pd.get_dummies(df, columns=cat_cols, dtype="int8")

df_encoded.head()


,id_lector,id_libro,rating,nombre,nacimiento,titulo,autor,editorial,anio_edicion,isbn,...,pais_republica checa,pais_rumania,pais_samoa americana,pais_san bartolome,pais_san pedro y miquelon,pais_serbia,pais_suiza,pais_turquia,pais_uruguay,pais_venezuela
0,yura,-incluso-el-olvido,6,yura,1996,",INCLUSO EL OLVIDO.","VITAS, ROMÁN ROLANDO",DEL CLÉ,2015.0,9789873755132.0,...,0,0,0,0,0,0,0,0,0,0
1,manucasoul,100-anos-de-pulgarcito-la-revista-donde-empezo...,10,manuel,1952,100 AÑOS DE PULGARCITO. La revista donde empez...,"GUIRAL, ANTONI",BRUGUERA,2021.0,9788402425171.0,...,0,0,0,0,0,0,0,0,0,0
2,aketza,1001-comics-que-hay-que-leer-antes-de-morir,8,aketza,1981,1001 COMICS QUE HAY QUE LEER ANTES DE MORIR,"GRAVETT, PAUL",GRIJALBO,2012.0,9788425347702.0,...,0,0,0,0,0,0,0,0,0,0
3,albertofg1,1080-recetas-de-cocina,9,alberto fernandez,1970,1080 RECETAS DE COCINA,"ORTEGA, SIMONE",ALIANZA,2023.0,9788411483582.0,...,0,0,0,0,0,0,0,0,0,0
4,trini,1080-recetas-de-cocina,10,trinidad sanz,1959,1080 RECETAS DE COCINA,"ORTEGA, SIMONE",ALIANZA,2023.0,9788411483582.0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Pasar a numérico, forzando errores a NaN
df_encoded["nacimiento"] = pd.to_numeric(df_encoded["nacimiento"], errors="coerce")


In [48]:
df_encoded.to_csv("qll_limpio_encoded.csv", index=False)
print("Dataset guardado")


Dataset guardado


### Random Forest básico

In [49]:
X = df_encoded.drop(columns=["rating", "id_lector", "id_libro", "nombre", "titulo", "editorial", "autor","isbn"])
y = df_encoded["rating"]


In [50]:
non_numeric = X.select_dtypes(exclude=['number']).columns
print("Columnas con texto o no numéricas:")
print(non_numeric)

Columnas con texto o no numéricas:
Index([], dtype='object')


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=100,  
    max_depth=None,     
    n_jobs=-1,         
    random_state=42
)

rf.fit(X_train, y_train)


y_pred = rf.predict(X_test)


rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE inicial (Random Forest básico): {rmse:.4f}")


RMSE inicial (Random Forest básico): 1.8068
